# Distortion from transducer board
We made a measurement of the harmonic distortion in the output from the ultrahaptics board and out own self made board.

### Ultrahaptics board
- Low level access API using constant amplitude of 1.
- Square driving signal between 0 and +20 V at 40 kHz.

### Self-made board
- 16 transducers in parallel connection, in a 4x4 grid.
- Sinusoidal driving signal at 40 kHz.
- Driven by audio amplifier giving 20 V amplitude.

### Measuement setup
- Extends to 500 kHz.
- Microphone is calibrated to 140 kHz, so the amplitudes above that are not precise.


In [ ]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from configparser import ConfigParser
folder = '../Measurements/Distortion comparison/'
config = ConfigParser()

dB = lambda x: 20*np.log10(np.abs(x)/20e-6*2**0.5) 
# Extra scaling to calculate SPL (RMS) from single sided spectrum
dBmax = 135
dBmin = 15
fMin = 1e3
fMax = 500e3

def processData(name):
    data = pd.read_csv(folder + name +'/F1.txt',delimiter='\t')
    data.drop('Signal',axis=1,inplace=True)
    data.rename(columns={'Microphone':'Signal'},inplace=True)
    config.read(folder + name + '/settings.ini')
    n=len(data)
    fs=float(config['General']['fs'])
    win = signal.windows.boxcar(n)
    win = signal.windows.flattop(n)
    calibration = float(config['General']['Microphone calibration'])
    data=data/calibration
    data = pd.concat([data, 
                      data.rename(columns={'Signal':'Spectrum'}).
                      apply(lambda x:fft(x*win)/win.sum(),raw=True)],axis=1)
    data['t'] = np.arange(n)/fs
    data['f'] = fftfreq(n,1/fs)
    return data

def levelAt(data,f):
    fIdx = (data['f']>f*0.9) & (data['f']<f*1.1)
    return np.max(dB(data['Spectrum'][fIdx]))

In [ ]:
UH1trans = processData('UH single trans max amp')
singletrans = processData('Single trans max amp')
UH16trans = processData('UH 16 trans max amp')
proto16trans = processData('Board16 max amp') 

In [ ]:
UH1trans40k = '{:.1f}'.format(levelAt(UH1trans,40e3))
UH1trans80k = '{:.1f}'.format(levelAt(UH1trans,80e3)-levelAt(UH1trans,40e3))
UH1trans120k = '{:.1f}'.format(levelAt(UH1trans,120e3)-levelAt(UH1trans,40e3))
UH1trans160k = '{:.1f}'.format(levelAt(UH1trans,160e3)-levelAt(UH1trans,40e3))
singletrans40k = '{:.1f}'.format(levelAt(singletrans,40e3))
singletrans80k = '{:.1f}'.format(levelAt(singletrans,80e3)-levelAt(singletrans,40e3))
singletrans120k = '{:.1f}'.format(levelAt(singletrans,120e3)-levelAt(singletrans,40e3))
singletrans160k = '{:.1f}'.format(levelAt(singletrans,160e3)-levelAt(singletrans,40e3))

## Single transducer
When emitting from a single transducer, the distortion is apparent up to the third harmonic.
According to the theory for parametric speakers we would only expect the first harmonic.
From this it is clear that it is more distortion from the Ultrahaptics board than from our self-made board.

It is also worth noting the noise spread around the carrier from the Ultrahaptics board. 
This noise will modulate with the carrier to produce audible noise output.

Table:  Comparison between the Ultrahaptics board and a single transducer from our self-made board.
The values for the harmonics are relative to the carrier level.

Frequency  |  UH board                   |  Self-made board    
---------  |  --------------             |  ----------------
40 kHz     |   {{UH1trans40k}} dB SPL    |    {{singletrans40k}} dB SPL
80 kHz     |   {{UH1trans80k}} dB        |    {{singletrans80k}} dB
120 kHz    |   {{UH1trans120k}} dB       |    {{singletrans120k}} dB
160 kHz    |   {{UH1trans160k}} dB       |    {{singletrans160k}} dB


In [ ]:
fIdx = (UH1trans['f']>fMin) & (UH1trans['f']<fMax)
plt.plot(UH1trans['f'][fIdx],dB(UH1trans['Spectrum'][fIdx]),label='Ultrahaptics')

fIdx = (singletrans['f']>fMin) & (singletrans['f']<fMax)
plt.plot(singletrans['f'][fIdx],dB(singletrans['Spectrum'][fIdx]),label='Self-made')

plt.legend()
plt.title('Single transducer')

plt.ylim(ymin=dBmin,ymax=dBmax)
plt.ylabel('dB SPL')
plt.xlabel('f/Hz')
plt.show()

In [ ]:
UH16trans40k = '{:.1f}'.format(levelAt(UH16trans,40e3))
UH16trans80k = '{:.1f}'.format(levelAt(UH16trans,80e3)-levelAt(UH16trans,40e3))
UH16trans120k = '{:.1f}'.format(levelAt(UH16trans,120e3)-levelAt(UH16trans,40e3))
UH16trans160k = '{:.1f}'.format(levelAt(UH16trans,160e3)-levelAt(UH16trans,40e3))
proto16trans40k = '{:.1f}'.format(levelAt(proto16trans,40e3))
proto16trans80k = '{:.1f}'.format(levelAt(proto16trans,80e3)-levelAt(proto16trans,40e3))
proto16trans120k = '{:.1f}'.format(levelAt(proto16trans,120e3)-levelAt(proto16trans,40e3))
proto16trans160k = '{:.1f}'.format(levelAt(proto16trans,160e3)-levelAt(proto16trans,40e3))

## 16 transducers
Using 16 transducers the distortion is present to much higher orders, and the relative levels of the harmonics are higher.
The difference between the Ultrahaptics board and out self-made bord is smaller here. This could be due to higher distortion in the audio amplifier.

Table:  Comparison between the Ultrahaptics board and 16 transducers from our self-made board.
The values for the harmonics are relative to the carrier level.

Frequency  |  UH board                   |  Self-made board    
---------  |  --------------             |  ----------------
40 kHz     |   {{UH16trans40k}} dB SPL    |    {{proto16trans40k}} dB SPL
80 kHz     |   {{UH16trans80k}} dB        |    {{proto16trans80k}} dB
120 kHz    |   {{UH16trans120k}} dB       |    {{proto16trans120k}} dB
160 kHz    |   {{UH16trans160k}} dB       |    {{proto16trans160k}} dB

In [ ]:
fIdx = (UH16trans['f']>fMin) & (UH16trans['f']<fMax)
plt.plot(UH16trans['f'][fIdx],dB(UH16trans['Spectrum'][fIdx]),label='Ultrahaptics')

fIdx = (proto16trans['f']>fMin) & (proto16trans['f']<fMax)
plt.plot(proto16trans['f'][fIdx],dB(proto16trans['Spectrum'][fIdx]),label='Self-made')

plt.legend()
plt.title('16 transducers')

plt.ylim(ymin=dBmin,ymax=dBmax)
plt.ylabel('dB SPL')
plt.xlabel('f/Hz')
plt.show()

## Summary
- There is more distortion in all measurements than what is expencted from the theory for parametric loudspekers.
- It is problematic to know where the distortion is coming from. It could be from the driving signal to the transducer (e.g. harmonics in the square wave or distortion in the amplifier), saturation effects in the transducer, or additional nonlinearities in air.
- There is more distortion in the output from the Ultrahaptics board than from our self-made simple board.

We cannot see the difference between the internal distortions in the hardware and the distortions from the nonlinearties in air. 
This makes it problematic to measure and idetify the nonlinerity in air. We would need this in order to design preprocessing methods for the parametric sound.
It will also be good to have a deeper knowledge about the nonlinearity in air when combining the different modalities.

A related issue is the measurement range. Since we cannot be sure of the exact levels of the higher harmonics, it will not be possible to use single tone measurements to identify the nonlinearities. A workaround for this is to use multiple tones in the ultrasonic range as the input to the transducers, which will modulate and create audible sound. By measuring this audible sound it is in principle possible to identify the nonlinearity.
If the input signal is not sufficiently free from distortion, this identification method becomes too complicated to perform in practice.

This is the reason that we need more control over the signals driving the transducers, and better understanding of what the board actually outputs given certain input values.